# Data Extraction

This notebook handles the loading, cleaning, and initial structuring of data from the zebrafish behavioral study for further analysis


## Brief Documentation

**General:**
- **fish_id**: Unique identifier for each fish.
- **battery_suite**: Identifies each battery suite. For further documentation, check "Defining Battery Suites" section.
- **group**: Groupes fish by treatment and exposure simultaneously.
- **date**: Date of the experiment.
- **start_time**: Exact time each fish started the battery suite.
- **treatment**: Concentration of ethanol in percentage (0%, 0.5% and 1%).
- **exposure**: Duration in hours fish were exposed to ethanol.

**Light-Dark Box Parameters:**
- **latency**: Time it took for the fish to choose between bright or dark environment.
- **first_choice**: Where the fish decided to go first. Legend is: "C" stands for "Claro" (bright) and "E" for "Escuro" (dark).
- **num_changes**: Number of times fish switched between bright and dark environments.
- **time_bright**: Total time (in seconds) spent in the bright.
- **mtpc** a.k.a **"Mean Time Per Change"**: average time (in seconds) fish spent on the bright environment per time it went there.

**Novel Tank Parameters:**
- **mov_bottom**: Time (in sec) the fish explored the bottom zone of the tank.
- **mov_upper**: Time (in sec) the fish explored the upper zone of the tank.
- **mov_total**: Total time (in sec) the fish explored the tank (derived).
- **dist_bottom**: Distance (in cm) traveled in the bottom zone.
- **dist_upper**: Distance (in cm) traveled in the upper zone.
- **dist_total**: Total distance (in cm) traveled by the fish during novel tank test.
- **vel_bottom**: Average velocity (cm/s) in the bottom zone.
- **vel_upper**: Average velocity (cm/s) in the upper.
- **vel_mean**: Average velocity (cm/s) overall during the novel tank test.

**Endurance Test Parameters:**
- **attempts**: Number of attempts to respond and resist the first water flow.
- **last_flux**: Last flow rate (L/min) the fish sustained.
- **time_in_last_flux**: Time (in seconds) spent enduring the last flow rate.
- **resistance_index**: Indicates performance of the fish in the endurance test. For further information, check next sections.

**Physiology and Biometrics:**
- **lt**: Total length of the fish (in cm).
- **ls** a.k.a **Length Standard**: Length (in cm) of the physiologically active part of the fish body, it excludes tail length.
- **wt**: Weight of the fish (in grams).
- **Kc** a.k.a **Fulton’s Condition Factor**: An index that reflects fish nutritional state.
- **sex**: Biological sex of the fish (male, female, or undetermined).
- **blood_sugar**: Blood glucose levels (in mg/dL) measured right after the end of the battery suite.

**Physicochemical:**
- **conductivity**: Water conductivity (in µS/cm) measured in exposure and nesting tanks.
- **pH**: pH of water in exposure and nesting tanks.

## Summary of Dataframes:

- **draft:** original dataframe containing all data collected
- **draft1:** dataframe for battery suite 1
- **draft2:** dataframe for battery suite 2
- **draft3:** dataframe for battery suite 3

## Libraries and Settings

In [1]:
# Libraries 📚 
import os
import numpy as np
import pandas as pd
import re

# Settings ⚙️ 
pd.set_option('display.max_columns', None)

## Functions

In [2]:
def resistance_index(row):    
    if pd.isna(row['last_flux']) or pd.isna(row['time_in_last_flux']):
        return None
    flux_sum = sum(range(1, int(row['last_flux'])))
    last_flux_weight = row['last_flux'] * row['time_in_last_flux'] / 60    
    return flux_sum + last_flux_weight



def vel_mean(row):
    if pd.isna(row['vel_bottom']) and pd.isna(row['vel_upper']):
        return 0
    elif row['vel_bottom'] == 0 or pd.isna(row['vel_bottom']):
        return row['vel_upper']
    elif row['vel_upper'] == 0 or pd.isna(row['vel_upper']):
        return row['vel_bottom']
    else:
        return (row['vel_bottom'] + row['vel_upper']) / 2

def get_mtpc(row):
    if row['time_bright'] == 0:
        return 0
    elif row['num_changes'] == 0:
        return row['time_bright']
    else:
        return row['time_bright'] / row['num_changes']



def drop_fish(df, fish_id):
    return df.drop(df[df['fish_id'] == fish_id].index)        

## Extracting the Data

In [3]:
draft = pd.read_csv('masters_data.csv')

In [4]:
draft['datetime'] = draft.date + ' ' + draft.start_time
draft['datetime'] = draft['datetime'].apply(lambda x: re.sub(r'\s+', '', str(x)))
draft['datetime'] = draft['datetime'].apply(lambda x: x[:10] + ' ' + x[10:] if isinstance(x, str) else x)
draft.loc[:277, 'datetime'] = pd.to_datetime(draft.loc[:277, 'datetime'])
draft = draft.drop(columns=['date', 'start_time'])
datetime_col = draft.pop('datetime')
draft.insert(5, 'datetime', datetime_col)

In [5]:
clean_first_choice = {
    'C' : 'bright',
    'E' : 'dark'
}

draft.first_choice = draft.first_choice.replace(clean_first_choice)

In [6]:
clean_group = {
 '96h_CTR': '96h_0.0',
 '24h_CTR': '24h_0.0',
 '01h_CTR': '01h_0.0',
 '1h_CTR': '01h_0.0',
 '1h_0.5' : '01h_0.5',
 '1h_1.0' : '01h_1.0'}

draft.group = draft.group.replace(clean_group)

In [7]:
clean_group_2 = {
    '01h_0.0': '01h 0.0%',
    '01h_0.5': '01h 0.5%',
    '01h_1.0': '01h 1.0%',
    '24h_0.0' : '24h 0.0%',
    '24h_0.5' : '24h 0.5%',
    '24h_1.0' : '24h 1.0%',
    '96h_0.0' : '96h 0.0%',
    '96h_0.5' : '96h 0.5%',
    '96h_1.0' : '96h 1.0%'
}

draft.group = draft.group.replace(clean_group_2)

In [8]:
clean_sex = {
    'm' : 'male',
    'f' : 'female'
}

draft.sex = draft.sex.replace(clean_sex)

In [9]:
draft['sex'] = draft['sex'].fillna('unidentified')

In [59]:
draft.shape

(414, 32)

## Creating Derivated Parameters

- `mtpc` = `time_bright` / `num_changes`
  
- `mov_total` = `mov_bottom` + `mov_upper`
  
- `dist_total` = `dist_bottom` + `dist_upper`
  
- `vel_bottom` = `dist_bottom` / `dist_upper`
  
- `vel_upper` = `dist_upper` / `mov_upper`
  
- `vel_total` = (`vel_bottom` + `vel_upper`) / 2

In [10]:
draft['mtpc'] = draft.apply(get_mtpc, axis=1)

In [11]:
draft['mov_total'] = draft.mov_bottom + draft.mov_upper

In [12]:
draft['dist_total'] = draft.dist_bottom + draft.dist_upper

In [13]:
draft['vel_bottom'] = draft.dist_bottom / draft.mov_bottom

In [14]:
draft['vel_upper'] = draft.dist_upper / draft.mov_upper

In [15]:
draft['vel_mean'] = draft.apply(vel_mean, axis=1)

### Fulton's Condition Factor

The **Fulton's Condition Factor** $(K_c)$ is a measure used to assess the health and well-being of fish based on their weight and length. It points out to the fish's nutritional status and general condition, reflecting if they are, for example, underweight, well-fed, or obese.

$$
Factor = \frac{Weigth}{Length^3} \times 100
$$

In [16]:
# Fulton's Condition Factor (Kc)
draft['Kc'] = (draft['wt'] / (draft['ls'] ** 3)) * 100

### Resistance Index

The **Resistance Index** is a measure of a fish's ability to swim against a water current. It reflects the fish's stamina and resistance to the flow of water. The test involves gradually increasing water flow and recording how long the fish can maintain swimming against it. The index is calculated by summing the fluxes the fish can resist and adding a weighted contribution from the last flux sustained by the fish.

$$
SRI = \sum \text{fluxes sustained for a whole minute} + \left( \text{last flux} \times \frac{\text{time in last flux (s)}}{60} \right)
$$

In [17]:
draft['resistance_index'] = draft.apply(resistance_index, axis=1)

## Defining Battery Suites

- **Battery Suite 1 (BT1)** consists of three tests: light/dark box, novel tank, and resistance test. These tests are applied sequentially to assess anxiety-like behaviors, locomotor activity and physical endurance.
  
- **Battery Suite 2 (BT2)** changes the order of the first two tests from BT1. The novel tank test is performed first, followed by the light/dark box test. This sequence is designed to check the performance of the novel tank test as a standalone measure and the light/dark box test as a sequential test, examining how previous test exposure influences behavior.
  
- **Battery Suite 3 (BT3)** focuses solely on the resistance test. This suite is designed to compare the performance of the resistance test when it is conducted alone, as opposed to being part of the full suite in BT1, allowing for a clearer understanding of its impact when not influenced by the previous tests.

In [18]:
draft1 = draft[draft['battery_suite'] == 1]

In [19]:
# dropping redundant column
draft1 = draft1.drop(columns='battery_suite')

In [20]:
draft2 = draft[draft['battery_suite'] == 2]

In [21]:
# parameters from Resistance Index must be dropped from battery suite 2
draft2 = draft2.drop(columns=['battery_suite', 'attempts', 'last_flux', 'time_in_last_flux', 'resistance_index'])

In [22]:
draft3 = draft[draft['battery_suite'] == 3]

In [23]:
# addressing battery suite 3 competent columns
draft3 = draft3[['fish_id', 'group', 'treatment', 'exposure', 'attempts', 'last_flux', 'time_in_last_flux', 'resistance_index', 'lt', 'ls', 'wt', 'Kc', 'sex', 'blood_sugar']]

# Initial Cleaning and Structuring

## Checking and Addressing NaN values

### Battery Suite 1

In [24]:
draft1.isna().sum()

fish_id               0
group                 0
treatment             0
exposure              0
datetime              0
latency               1
first_choice          1
num_changes           2
time_bright           2
mtpc                  2
mov_bottom            1
mov_upper             1
mov_total             1
dist_bottom           1
dist_upper            1
dist_total            1
vel_bottom            5
vel_upper            11
vel_mean              0
attempts              1
last_flux             1
time_in_last_flux     1
resistance_index      1
lt                    0
ls                    0
wt                    0
Kc                    0
sex                   0
blood_sugar          27
conductivity          0
pH                    0
dtype: int64

---

Light-Dark Box

---

In [25]:
draft1.loc[draft1.latency.isna()].fish_id # fish jumped from central compartment to the bright side before acclimatation ended
# might as well consider dropping

91    1BT24h01015
Name: fish_id, dtype: object

In [26]:
draft1 = drop_fish(draft1, '1BT24h01015')

In [27]:
draft1.loc[draft1.first_choice.isna()].fish_id # fish remained the whole test in the central compartment
# might as well create a third value for fishes that didn't choose between bright or dark

29    1BT01h00516
Name: fish_id, dtype: object

In [28]:
draft1.loc[draft1.num_changes.isna()].fish_id # died during test, it can be dropped! (1BT24h00513)

73    1BT24h00513
Name: fish_id, dtype: object

In [29]:
draft1 = drop_fish(draft1, '1BT24h00513')

In [30]:
draft1.loc[draft1.mtpc.isna()].fish_id
# 1BT01hCTR10 went dark side and never changed
# 1BT01h00508 went dark side and never changed
# 1BT01h00509 went dark side and never changed
# 1BT01h00516 didn't choose between dark of bright environment
# 1BT01h01007 went dark side and never changed
# 1BT01h01012 went dark side and never changed
# 1BT96hCTR10 went dark side and never changed

Series([], Name: fish_id, dtype: object)

---

Novel Tank

---

In [31]:
draft1.loc[draft1.vel_bottom.isna()].fish_id
# 1BT01h00508 freezing
# 1BT01h00509 freezing
# 1BT01h00510 freezing
# 1BT01h01012 freezing

21    1BT01h00508
22    1BT01h00509
23    1BT01h00510
41    1BT01h01012
Name: fish_id, dtype: object

In [32]:
draft1.vel_bottom = draft1.vel_bottom.fillna(0) # correctly representing freezing as velocity 0

In [33]:
draft1.loc[draft1.vel_upper.isna()].fish_id
# here we want to keep the NaNs as they represent non-exploration of upper portion

1     1BT01hCTR02
11    1BT01hCTR12
21    1BT01h00508
22    1BT01h00509
23    1BT01h00510
34    1BT01h01005
37    1BT01h01008
41    1BT01h01012
48    1BT24hCTR03
83    1BT24h01007
Name: fish_id, dtype: object

---

Biometrics

---

In [34]:
draft1.blood_sugar.isna().sum()
# might as well fill in with means (based on group)

25

In [35]:
draft1.blood_sugar = draft1.groupby('group')['blood_sugar'].transform(lambda x: x.fillna(x.mean()))

Remaning NaNs for this subset are actually data, meaning inanimity, which is useful and should be dealt differently depending on the question we are asking. Therefore, I will leave them.

In [36]:
draft1.to_csv('battery_suite_1.csv', index=True) # I want to keep the original index

### Battery Suite 2

In [37]:
draft2.isna().sum()

fish_id           0
group             0
treatment         0
exposure          0
datetime          0
latency           0
first_choice      1
num_changes       1
time_bright       1
mtpc              1
mov_bottom        1
mov_upper         1
mov_total         1
dist_bottom       1
dist_upper        1
dist_total        1
vel_bottom       10
vel_upper        17
vel_mean          0
lt                2
ls                2
wt                2
Kc                2
sex               0
blood_sugar     103
conductivity     32
pH               32
dtype: int64

---

Light-Dark Box

---

In [38]:
draft2.loc[draft2.first_choice.isna()].fish_id
# 2BT96h01014 fish never made a choice

275    2BT96h01014
Name: fish_id, dtype: object

In [39]:
draft2.loc[draft2.num_changes.isna()].fish_id 
# fish never made a choice

275    2BT96h01014
Name: fish_id, dtype: object

In [40]:
draft2.loc[draft2.time_bright.isna()].fish_id
# fish never made a choice

275    2BT96h01014
Name: fish_id, dtype: object

In [41]:
draft2.time_bright = draft2.time_bright.fillna(0)

In [42]:
draft2.loc[draft2.mtpc.isna()].fish_id
# 2BT01h00502 instantly went darkside and never came back
# 2BT01h00504 instantly went darkside and never came back
# 2BT01h01007 instantly went darkside and never came back
# 2BT24hCTR02 instantly went darkside and never came back
# 2BT24h00506 instantly went darkside and never came back
# 2BT24h00511 instantly went darkside and never came back
# 2BT24h00513 instantly went darkside and never came back
# 2BT96h01014 never made a choice

275    2BT96h01014
Name: fish_id, dtype: object

---

Novel Tank

---

In [43]:
draft2.loc[draft2.mov_bottom.isna()]
# data on this fish is curiously missing, there are no notes on this. We might need to drop it
# since there is data recorded for other tests, we might as well fill with mean values)))))))))

,fish_id,group,treatment,exposure,datetime,latency,first_choice,num_changes,time_bright,mtpc,mov_bottom,mov_upper,mov_total,dist_bottom,dist_upper,dist_total,vel_bottom,vel_upper,vel_mean,lt,ls,wt,Kc,sex,blood_sugar,conductivity,pH
202,2BT24h00501,24h 0.5%,0.5,24,2021-08-11 10:00:30,13.0,bright,16.0,444.0,27.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.5,2.0,0.17,2.125,male,NaN,110.0,7.5


In [44]:
draft2.loc[draft2.vel_bottom.isna()].fish_id
# all fishes here explored just the upper portion.

202    2BT24h00501
204    2BT24h00503
205    2BT24h00504
206    2BT24h00505
210    2BT24h00509
211    2BT24h00510
216    2BT24h00515
217    2BT24h00516
265    2BT96h01004
275    2BT96h01014
Name: fish_id, dtype: object

In [45]:
draft2.loc[draft2.vel_upper.isna()].fish_id
# all fish here but one explored just bottom portion

156    2BT01h00503
161    2BT01h00508
163    2BT01h00510
164    2BT01h00511
166    2BT01h00513
169    2BT01h00516
172    2BT01h01003
176    2BT01h01007
177    2BT01h01008
178    2BT01h01009
180    2BT01h01011
186    2BT24hCTR01
202    2BT24h00501
204    2BT24h00503
226    2BT24h01009
229    2BT24h01012
275    2BT96h01014
Name: fish_id, dtype: object

---

Biometrics

---

In [46]:
draft2['lt'] = draft2['lt'].fillna(draft2['lt'].mean())

In [47]:
draft2.ls = draft2['ls'].fillna(draft2['ls'].mean())

In [48]:
draft2.wt = draft2['wt'].fillna(draft2['wt'].mean())

In [49]:
draft2['Kc'] = (draft2['wt'] / (draft2['ls'] ** 3)) * 100

In [50]:
draft2.blood_sugar = draft2.groupby('group')['blood_sugar'].transform(lambda x: x.fillna(x.mean()))

In [51]:
draft2.to_csv('battery_suite_2.csv', index=True)

### Battery Suite 3

In [52]:
draft3.isna().sum()

fish_id               0
group                 0
treatment             0
exposure              0
attempts              1
last_flux             1
time_in_last_flux     1
resistance_index      1
lt                    1
ls                    1
wt                    1
Kc                    1
sex                   0
blood_sugar          63
dtype: int64

In [53]:
draft3.loc[draft3.attempts.isna()]

,fish_id,group,treatment,exposure,attempts,last_flux,time_in_last_flux,resistance_index,lt,ls,wt,Kc,sex,blood_sugar
324,3BT01h01015,01h 1.0%,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unidentified,NaN


In [54]:
draft3 = drop_fish(draft3, '3BT01h01015')

In [55]:
draft3.blood_sugar = draft3.groupby('group')['blood_sugar'].transform(lambda x: x.fillna(x.mean()))

In [56]:
draft3.to_csv('battery_suite_3.csv', index=True)